In [8]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from collections import defaultdict

# pd.options.display.float_format = "{:,.4f}".format

# import matplotlib.pyplot as plt
# import seaborn as sns

# import statsmodels.api as sm
# from scipy.stats import norm
# from sklearn.linear_model import LinearRegression
# from arch import arch_model

# import warnings
# warnings.filterwarnings("ignore")

# import importlib
# import utils as ut
# importlib.reload(ut)

--- Day 1: Historian Hysteria ---

In [4]:
file_path = './data/input_1.txt'
data = np.loadtxt(file_path, dtype=int)

left, right = np.sort(data[:, 0]), np.sort(data[:, 1])

np.sum(np.abs(left - right))

np.int64(2086478)

In [5]:
counter = Counter(right)
sum(counter[l] * l for l in left)

np.int64(24941624)

--- Day 2: Red-Nosed Reports ---

In [26]:
file_path = './data/input_2.txt'
with open(file_path, 'r') as file:
    data = [list(map(int, line.split())) for line in file]

def is_safe(row):
    differences = [row[i] - row[i - 1] for i in range(1, len(row))]
    return all(1 <= diff <= 3 for diff in differences) or all(-3 <= diff <= -1 for diff in differences)

sum(is_safe(row) for row in data)

483

In [31]:
def is_removable_safe(row):
    if is_safe(row):
        return True
    for i in range(len(row)):
        new_row = row[:i] + row[i + 1:]
        if is_safe(new_row):
            return True
    return False

sum(is_removable_safe(row) for row in data)

528

--- Day 3: Mull It Over ---

In [10]:
file_path = './data/input_3.txt'
with open(file_path, 'r') as file:
    data = file.read()

pattern = r'mul\((\d+),(\d+)\)'
matches = re.findall(pattern, data)
sum(int(a) * int(b) for a, b in matches)

175615763

In [22]:
mul_pattern = r"mul\((\d+),(\d+)\)"
do_pattern = r"do\(\)"
dont_pattern = r"don't\(\)"

pattern = f"{mul_pattern}|{do_pattern}|{dont_pattern}"

matches = re.finditer(pattern, data)

mul_enabled = True
total_sum = 0

for match in matches:
    if match.group().startswith('do()'):
        mul_enabled = True
    elif match.group().startswith("don't()"):
        mul_enabled = False
    elif mul_enabled and match.group().startswith("mul("):
        a, b = map(int, match.groups())
        total_sum += a * b

total_sum

74361272

--- Day 4: Ceres Search ---

In [62]:
file_path = './data/input_4.txt'
with open(file_path, 'r') as file:
    data = file.read().strip()
lines = data.splitlines()

In [72]:
def collect_all_directions(grid):
    R, C = len(grid), len(grid[0])
    directions = []
    directions.extend(grid)
    directions.extend([row[::-1] for row in grid])
    directions.extend([''.join([grid[r][c] for r in range(R)]) for c in range(C)])
    directions.extend([''.join([grid[R - 1 - r][c] for r in range(R)]) for c in range(C)])

    for d in range(-R + 1, C):
        directions.append(''.join([grid[r][r - d] for r in range(max(0, d), min(R, C + d))]))
    for d in range(-R + 1, C):
        directions.append(''.join([grid[R - 1 - r][r - d] for r in range(max(0, d), min(R, C + d))]))
    for d in range(-R + 1, C):
        directions.append(''.join([grid[r][C - 1 - (r - d)] for r in range(max(0, d), min(R, C + d))]))
    for d in range(-R + 1, C):
        directions.append(''.join([grid[R - 1 - r][C - 1 - (r - d)] for r in range(max(0, d), min(R, C + d))]))

    return directions

directions = collect_all_directions(lines)
sum(direction.count('XMAS') for direction in directions)

2427

In [77]:
def count_X_MAS(grid):
    res = 0
    for r in range(1, len(grid) - 1):
        for c in range(1, len(grid[0]) - 1):
            if grid[r][c] == 'A':
                if set([grid[r - 1][c - 1], grid[r + 1][c + 1]]) == {'M', 'S'} and set([grid[r - 1][c + 1], grid[r + 1][c - 1]]) == {'M', 'S'}:
                    res += 1

    return res
count_X_MAS(lines)

1900

--- Day 5: Print Queue ---

In [48]:
file_path = './data/input_5.txt'
rules, pages = [], []

with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if '|' in line:
            rules.append(list(map(int, line.split('|'))) )
        elif ',' in line:
            pages.append(list(map(int, line.split(','))) )

In [49]:
rules_dict = defaultdict(list)
for a, b in rules:
    rules_dict[b].append(a)

def valid_middle(numbers):
    seen, forbidden = set(), set()
    for number in numbers:
        if (number in forbidden) and (number not in seen):
            return 0
        seen.add(number)
        forbidden.update(rules_dict[number])

    return numbers[len(numbers) // 2]

sum(valid_middle(numbers) for numbers in pages)

5087

In [64]:
def reorder_invalid_middle(numbers):
    seen, forbidden = set(), set()
    mid_ind = len(numbers) // 2
    for number in numbers:
        if (number in forbidden) and (number not in seen):
            reordered = sorted(numbers, key=lambda x: len([y for y in rules_dict[x] if y in numbers]))
            return reordered[mid_ind]
        seen.add(number)
        forbidden.update(rules_dict[number])
    return 0

sum(reorder_invalid_middle(numbers) for numbers in pages)

4971

--- Day 6: Guard Gallivant ---